# 01 - Inspect and Clean Data

This notebook is used to explore and clean the raw datasets for FindMyBorough.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def load_and_preview(file_path, sheet=None, header=0):
    """
    Load and preview a dataset (.csv or .xls/.xlsx).
    
    Parameters:
    - file_path (str): Path to the dataset file.
    - sheet (int or str, optional): Sheet index or name for Excel files.
    - header (int): Row index to use as column names for Excel/CSV.

    Returns:
    - pd.DataFrame
    """
    print(f"Loading file: {file_path}")
    
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, header=header)
        
    elif file_path.endswith(".xls") or file_path.endswith(".xlsx"):
        xls = pd.ExcelFile(file_path)
        print("Available sheets:", xls.sheet_names)
        
        if sheet is None:
            sheet = 0  # Default to first sheet
        
        df = xls.parse(sheet, header=header)
        
    else:
        raise ValueError("Unsupported file type. Use .csv or .xls/.xlsx.")
    
    print("\nPreview:")
    display(df.head())
    print("\nInfo:")
    display(df.info())
    print("\nColumns:", df.columns.tolist())
    
    return df


In [3]:
def standardize_borough_names(series):
    """
    Normalize borough names:
    - Lowercase
    - Strip whitespace
    - Replace '&' with 'and'
    - Replace known variants (only if needed)
    """
    replacements = {
        'westminster city': 'westminster'
    }

    return (
        series
        .astype(str)
        .str.strip()
        .str.lower()
        .str.replace('&', 'and')
        .replace(replacements)
    )

In [4]:
valid_boroughs = [
    'barking and dagenham', 'barnet', 'bexley', 'brent', 'bromley', 'camden', 'city of london',
    'croydon', 'ealing', 'enfield', 'greenwich', 'hackney', 'hammersmith and fulham',
    'haringey', 'harrow', 'havering', 'hillingdon', 'hounslow', 'islington',
    'kensington and chelsea', 'kingston upon thames', 'lambeth', 'lewisham', 'merton',
    'newham', 'redbridge', 'richmond upon thames', 'southwark', 'sutton',
    'tower hamlets', 'waltham forest', 'wandsworth', 'westminster'
]

def filter_valid_boroughs(df, borough_column='borough', valid_boroughs=None):
    """
    Filter a DataFrame to include only valid (normalized) London boroughs.
    
    Parameters:
    - df: pd.DataFrame — the dataset to filter
    - borough_column: str — name of the column containing borough names
    - valid_boroughs: list[str] — list of valid boroughs in lowercase

    Returns:
    - pd.DataFrame — filtered DataFrame
    """
    if valid_boroughs is None:
        raise ValueError("valid_boroughs list is required")

    df[borough_column] = df[borough_column].str.strip().str.lower()
    
    return df[df[borough_column].isin(valid_boroughs)].copy()


## Rental Prices

In [5]:
df_rent = load_and_preview("../data/raw/voa-average-rent-borough.xls", sheet="Raw data", header=2)

Loading file: ../data/raw/voa-average-rent-borough.xls
Available sheets: ['Metadata', 'Summary', 'Pivot Table', 'Raw data']

Preview:


,Year,Quarter,Code,Area,Category,Count of rents,Average,Lower quartile,Median,Upper quartile
0,2011,Q2,E09000001,City of London,Room,-,-,-,-,-
1,2011,Q2,E09000002,Barking and Dagenham,Room,92,336,282,347,390
2,2011,Q2,E09000003,Barnet,Room,945,450,399,433,500
3,2011,Q2,E09000004,Bexley,Room,119,390,347,390,433
4,2011,Q2,E09000005,Brent,Room,344,469,390,457,550



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6160 entries, 0 to 6159
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            6160 non-null   int64 
 1   Quarter         6160 non-null   object
 2   Code            6160 non-null   object
 3   Area            6160 non-null   object
 4   Category        6160 non-null   object
 5   Count of rents  6160 non-null   object
 6   Average         6160 non-null   object
 7   Lower quartile  6160 non-null   object
 8   Median          6160 non-null   object
 9   Upper quartile  6160 non-null   object
dtypes: int64(1), object(9)
memory usage: 481.4+ KB


None


Columns: ['Year', 'Quarter', 'Code', 'Area', 'Category', 'Count of rents', 'Average', 'Lower quartile', 'Median', 'Upper quartile']


In [6]:
# Filter rows where the property type is 'All categories'
df_all = df_rent[df_rent['Category'] == 'All categories']

# Group by borough and calculate the median rent
df_rent_clean = df_all.groupby('Area')['Median'].median().reset_index()

# Rename columns to match the feature naming convention
df_rent_clean.columns = ['borough', 'avg_rent']

# Normalize borough names and filter only London ones
df_rent_clean['borough'] = standardize_borough_names(df_rent_clean['borough'])
df_rent_clean = filter_valid_boroughs(df_rent_clean, 'borough', valid_boroughs)
df_rent_clean.to_csv("../data/clean/clean_rent.csv", index=False)
df_rent_clean.head()

,borough,avg_rent
0,barking and dagenham,900.5
1,barnet,1264.0
2,bexley,850.0
3,brent,1314.0
4,bromley,1025.0


## Crime

In [7]:
df_crime = load_and_preview("../data/raw/MPS Monthly Crime Dashboard_BoroughSNT_TNOCrimeDatafy22-23_03.csv")

Loading file: ../data/raw/MPS Monthly Crime Dashboard_BoroughSNT_TNOCrimeDatafy22-23_03.csv

Preview:


,Month_Year,Area Type,Borough_SNT,Area name,Area code,Offence Group,Offence Subgroup,Measure,Financial Year,FY_FYIndex,Count
0,2022-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Arson and Criminal Damage,Criminal Damage,Offences,fy22-23,fy22-23_03,8
1,2022-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Arson and Criminal Damage,Criminal Damage,Outcomes,fy22-23,fy22-23_03,1
2,2022-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Burglary,Burglary Business and Community,Offences,fy22-23,fy22-23_03,1
3,2022-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Burglary,Burglary Business and Community,Outcomes,fy22-23,fy22-23_03,1
4,2022-04-01,Borough,Aviation Security(SO18),Aviation Security(SO18),SO18,Drug Offences,Possession of Drugs,Offences,fy22-23,fy22-23_03,2



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218229 entries, 0 to 218228
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Month_Year        218229 non-null  object
 1   Area Type         218229 non-null  object
 2   Borough_SNT       218229 non-null  object
 3   Area name         218229 non-null  object
 4   Area code         218229 non-null  object
 5   Offence Group     218229 non-null  object
 6   Offence Subgroup  218229 non-null  object
 7   Measure           218229 non-null  object
 8   Financial Year    218229 non-null  object
 9   FY_FYIndex        218229 non-null  object
 10  Count             218229 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 18.3+ MB


None


Columns: ['Month_Year', 'Area Type', 'Borough_SNT', 'Area name', 'Area code', 'Offence Group', 'Offence Subgroup', 'Measure', 'Financial Year', 'FY_FYIndex', 'Count']


In [8]:
# Keep only rows where the measure is 'Offences'
df_crime_filtered = df_crime[df_crime['Measure'] == 'Offences']

# Group by borough and sum total crimes
df_crime_clean = df_crime_filtered.groupby('Area name')['Count'].sum().reset_index()

# Rename columns
df_crime_clean.columns = ['borough', 'total_crimes']

# Normalize borough names and filter only London ones
df_crime_clean['borough'] = standardize_borough_names(df_crime_clean['borough'])
df_crime_clean = filter_valid_boroughs(df_crime_clean, 'borough', valid_boroughs)

# Save to CSV
df_crime_clean.to_csv("../data/clean/clean_crime.csv", index=False)
df_crime_clean.head()

,borough,total_crimes
17,barking and dagenham,24903
23,barnet,31986
44,bexley,18776
60,brent,34400
72,bromley,26028


## Age group

In [9]:
df_age = load_and_preview("../data/raw/londonboroughs.xlsx", sheet="Age London Boroughs")

Loading file: ../data/raw/londonboroughs.xlsx
Available sheets: ['Metadata', 'Household Type Boroughs', 'Ethnic Group Boroughs', 'London HH Type', 'Age London Boroughs', 'Age London', 'London Ethnic Group']

Preview:


,Lower tier local authorities Code,Lower tier local authorities,Age (18 categories) Code,Age (18 categories),Observation
0,E09000001,City of London,1,Aged 4 years and under,213
1,E09000001,City of London,2,Aged 5 to 9 years,159
2,E09000001,City of London,3,Aged 10 to 15 years,206
3,E09000001,City of London,4,Aged 16 to 19 years,184
4,E09000001,City of London,5,Aged 20 to 24 years,965



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 5 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Lower tier local authorities Code  594 non-null    object
 1   Lower tier local authorities       594 non-null    object
 2   Age (18 categories) Code           594 non-null    int64 
 3   Age (18 categories)                594 non-null    object
 4   Observation                        594 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 23.3+ KB


None


Columns: ['Lower tier local authorities Code', 'Lower tier local authorities', 'Age (18 categories) Code', 'Age (18 categories)', 'Observation']


In [10]:
df_age.columns = df_age.columns.str.strip()

df_age.rename(columns={
    "Lower tier local authorities": "borough",
    "Age (18 categories)": "age_group",
    "Observation": "population"
}, inplace=True)

df_age["borough"] = df_age["borough"].str.strip().str.lower()
df_age["age_group"] = df_age["age_group"].str.strip()

young_ages = [
    "Aged 20 to 24 years",
    "Aged 25 to 29 years",
    "Aged 30 to 34 years"
]

# Total population per borough
df_total_pop = df_age.groupby("borough")["population"].sum().reset_index()
df_total_pop.rename(columns={"population": "total_population"}, inplace=True)

# Young population per borough
df_young = df_age[df_age["age_group"].isin(young_ages)]
df_young_pop = df_young.groupby("borough")["population"].sum().reset_index()
df_young_pop.rename(columns={"population": "young_population"}, inplace=True)

# Merge and calculate ratio
df_age_features = pd.merge(df_total_pop, df_young_pop, on="borough", how="inner")
df_age_features["young_ratio"] = df_age_features["young_population"] / df_age_features["total_population"]

# Keep only final columns
df_age_features = df_age_features[["borough", "young_ratio"]]

In [15]:
# Normalize borough names and filter only valid London boroughs
df_age_features['borough'] = standardize_borough_names(df_age_features['borough'])
df_age_clean = filter_valid_boroughs(df_age_features, 'borough', valid_boroughs)

# Save cleaned version
df_age_clean.to_csv("../data/clean/clean_age.csv", index=False)

# Preview
df_age_clean.head()

,borough,young_ratio
0,barking and dagenham,0.212506
1,barnet,0.207044
2,bexley,0.184581
3,brent,0.246455
4,bromley,0.167963


## Zones

In [19]:
import pandas as pd

# Borough to zone mapping with lowercase keys
borough_zone_mapping = {
    "barking and dagenham": 4,
    "barnet": 4,
    "bexley": 4,
    "brent": 3,
    "bromley": 4,
    "camden": 1,
    "city of london": 1,
    "croydon": 4,
    "ealing": 3,
    "enfield": 4,
    "greenwich": 3,
    "hackney": 2,
    "hammersmith and fulham": 2,
    "haringey": 3,
    "harrow": 4,
    "havering": 4,
    "hillingdon": 4,
    "hounslow": 4,
    "islington": 2,
    "kensington and chelsea": 1,
    "kingston upon thames": 4,
    "lambeth": 2,
    "lewisham": 3,
    "merton": 4,
    "newham": 3,
    "redbridge": 4,
    "richmond upon thames": 4,
    "southwark": 2,
    "sutton": 4,
    "tower hamlets": 2,
    "waltham forest": 3,
    "wandsworth": 2,
    "westminster": 1,
}

df_zones = pd.DataFrame(list(borough_zone_mapping.items()), columns=["borough", "zone"])

df_zones.to_csv("../data/clean/borough_zones.csv", index=False)
print("borough_zones.csv saved.")

borough_zones.csv saved.


## Merge data

In [20]:
# Merge rent and crime data
df_features = pd.merge(df_rent_clean, df_crime_clean, on='borough', how='inner')

# Merge with age data
df_features = pd.merge(df_features, df_age_clean, on='borough', how='inner')

# Merge with zone data
df_zones = pd.read_csv("../data/clean/borough_zones.csv")
df_features = pd.merge(df_features, df_zones, on="borough", how="inner")

# Check for missing values
nulls = df_features[df_features.isnull().any(axis=1)]

if not nulls.empty:
    print("Warning: There are boroughs with missing values:")
    display(nulls)
else:
    print("No missing values found in merged dataset.")

# Normalize rent
# norm_rent: 1.0 means cheaper, 0.0 means more expensive
rent_min, rent_max = df_features['avg_rent'].min(), df_features['avg_rent'].max()
df_features['norm_rent'] = 1 - (df_features['avg_rent'] - rent_min) / (rent_max - rent_min)

# Normalize crime
# norm_safety: 1.0 means safer, 0.0 means more dangerous
crime_min, crime_max = df_features['total_crimes'].min(), df_features['total_crimes'].max()
df_features['norm_safety'] = 1 - (df_features['total_crimes'] - crime_min) / (crime_max - crime_min)

# Normalize age
# norm_youth: 1.0 means younger population, 0.0 means older
youth_min, youth_max = df_features['young_ratio'].min(), df_features['young_ratio'].max()
df_features['norm_youth'] = (df_features['young_ratio'] - youth_min) / (youth_max - youth_min)

# Normalize zone
# norm_centrality: 1.0 means central (zone 1), 0.0 means outer (zone 4)
zone_min, zone_max = df_features['zone'].min(), df_features['zone'].max()
df_features['norm_centrality'] = 1 - (df_features['zone'] - zone_min) / (zone_max - zone_min)


# Keep only final columns used for modeling
df_features = df_features[['borough', 'norm_rent', 'norm_safety', 'norm_youth', 'norm_centrality']]

# Save normalized feature set
df_features.to_csv("../data/clean/borough_features.csv", index=False)

# Preview
df_features.head(20)


No missing values found in merged dataset.


,borough,norm_rent,norm_safety,norm_youth,norm_centrality
0,barking and dagenham,0.964561,0.844750,0.258126,0.000000
1,barnet,0.709474,0.750410,0.233579,0.000000
2,bexley,1.0,0.926358,0.132636,0.000000
3,brent,0.674386,0.718257,0.410687,0.333333
4,bromley,0.877193,0.829766,0.057960,0.000000
5,camden,0.380351,0.654537,0.644201,1.000000
6,croydon,0.947368,0.649010,0.232035,0.000000
7,ealing,0.701754,0.708281,0.310613,0.333333
8,enfield,0.824561,0.718829,0.166822,0.000000
9,greenwich,0.780702,0.750783,0.406084,0.333333
